In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, mean_squared_error, plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.multiclass import OneVsRestClassifier, OutputCodeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

### CONSTANTS
SEED = 123
NUM_FEATURES = 19
TRAIN_PCT = 0.8

MAX_DEPTH = 4
MAX_ITER = 300
N_NEIGHBORS = 5



In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [4]:
def selectFeatures(x, y, train_size_pct=0.75):
    """
    selectFeatures
        x : The features of the dataset to be used for predictions
        y : The target class for each row in "x"
        train_size_pct : (default = 0.75) In the tange (0.0, 1.0), the ratio by which to split the data for training and testing
        @return (list) The names of the selected features
    """

    # Create classifiers
    rf = RandomForestClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    et = ExtraTreesClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    dectree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=SEED)

    classifier_mapping = {
        "RandomForest" : rf,
        "ExtraTrees" : et,
        "DecisionTree" : dectree
    }

    ### Split the dataset
    X_train_fs, X_test_fs, Y_train_fs, Y_test_fs = train_test_split(x, y, train_size=train_size_pct)

    model_features = {}

    for model_name, model in classifier_mapping.items():
        print(f'[Training] {model_name}')
        start_train = datetime.now()
        model.fit(X_train_fs, Y_train_fs)
        print(">>> Training Time: {}".format(datetime.now() - start_train))
        model_features[model_name] = model.feature_importances_
        model_score = model.score(X_test_fs, Y_test_fs)
        print(f'>>> Training Accuracy : {model_score*100.0}')
        print("")

    cols = X_train_fs.columns.values
    feature_df = pd.DataFrame({'features': cols})
    for model_name, model in classifier_mapping.items():
        feature_df[model_name] = model_features[model_name]

    ### Grab the nlargest features (by score) from each ensemble group
    all_f = []
    for model_name, model in classifier_mapping.items():
        try:
            all_f.append(feature_df.nlargest(NUM_FEATURES, model_name))
        except KeyError as e:
            print(f'*** Failed to add features for {model_name} : {e}')

    result = []
    for i in range(len(all_f)):
        result.extend(all_f[i]['features'].to_list())		# Concat the top nlargest scores from all groups into one list

    # selected_features contains the ensemble results for best features
    selected_features = list(set(result))					# Drop duplicate fields from the list

    return selected_features


    

In [5]:
def evaluateIndividualClassifiers(x, y, train_size_pct):
    """
    evaluateIndividualClassifiers
        x : The features of the dataset to be used for predictions
        y : The target class for each row in "x"
        train_size_pct : {float in the range(0.0, 1.0)} the percentage of the dataset that should be used for training
    """

    max_depth_x2 = MAX_DEPTH * 2
    max_iter_x2 = MAX_ITER * 2
    n_neighbors_x2 = N_NEIGHBORS * 2
    n_neighbors_d2 = N_NEIGHBORS // 2

    rf = RandomForestClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    rf_x2 = RandomForestClassifier(max_depth=max_depth_x2, criterion='entropy', random_state=SEED)
    et = ExtraTreesClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    dectree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=SEED)
    knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS)
    knn_x2 = KNeighborsClassifier(n_neighbors=n_neighbors_x2)
    knn_d2 = KNeighborsClassifier(n_neighbors=n_neighbors_d2)
    mlpnn = MLPClassifier(max_iter=MAX_ITER)
    mlpnnE = MLPClassifier(max_iter=MAX_ITER, early_stopping=True)
    mlpnn_x2 = MLPClassifier(max_iter=max_iter_x2)
    mlpnnE_x2 = MLPClassifier(max_iter=max_iter_x2, early_stopping=True)

    classifier_mapping = {
        f'RandomForest-{MAX_DEPTH}' : rf,
        f'RandomForest-{max_depth_x2}' : rf_x2,
        f'ExtraTrees-{MAX_DEPTH}' : et,
        f'DecisionTree-{MAX_DEPTH}' : dectree,
        f'KNeighbors-{N_NEIGHBORS}' : knn,
        f'KNeighbors-{n_neighbors_x2}' : knn_x2,
        f'KNeighbors-{n_neighbors_d2}' : knn_d2,
        f'MLP-{MAX_ITER}' : mlpnn,
        f'MLP-{MAX_ITER}-early' : mlpnnE,
        f'MLP-{max_iter_x2}' : mlpnn_x2,
        f'MLP-{max_iter_x2}-early' : mlpnnE_x2,
    }

    for model_name, model in classifier_mapping.items():

        train_test_model(model_name, model, x, y, train_size_pct)


In [6]:
def train_test_model(model_name, model, x, y, train_size_pct):

    # Split the data
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=train_size_pct)

    # Training
    print(f'\n[Training] {model_name}')
    start_train = datetime.now()
    model.fit(X_train, Y_train)
    print(f'>>> Training time: {datetime.now() - start_train}')

    ### Analyze Training
    train_acc = model.score(X_train, Y_train)
    print(f'>>> Training accuracy: {train_acc}')

    ### Testing
    start_predict = datetime.now()
    y_pred = model.predict(X_test)
    print(f'>>> Testing time: {datetime.now() - start_predict}')

    ### Analyze Testing
    calculateMetrics(Y_test, y_pred)



In [7]:
def calculateMetrics(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro", zero_division=0)
    mse = mean_squared_error(y_test, y_pred)
    f1score = f1_score(y_pred, y_test, average='weighted')
    print(">>> Metrics")
    print(f'- Accuracy  : {acc}')
    print(f'- Recall    : {recall}')
    print(f'- Precision : {precision}')
    print(f'- MSE       : {mse}')
    print(f'- F1 Score  : {f1score}')

    return [round(acc, 6), round(recall, 6), round(precision, 6), round(mse, 6), round(f1score, 6)]

In [8]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

Number of Rows: 2660377
Number of Columns: 85


,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,...,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Fwd Avg Bytes/Bulk,Fwd Avg Packets/Bulk,Fwd Avg Bulk Rate,Bwd Avg Bytes/Bulk,Bwd Avg Packets/Bulk,Bwd Avg Bulk Rate,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,192.168.10.5-104.16.207.165-54865-443-6,104.16.207.165,443,192.168.10.5,54865,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,33,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,192.168.10.5-104.16.28.216-55054-80-6,104.16.28.216,80,192.168.10.5,55054,6,7/7/2017 3:30,109,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,1.100917e+05,18348.62385,109.0,0.0,109.0,109.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,192.168.10.5-104.16.28.216-55055-80-6,104.16.28.216,80,192.168.10.5,55055,6,7/7/2017 3:30,52,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,2.307692e+05,38461.53846,52.0,0.0,52.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,29,256,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,192.168.10.16-104.17.241.25-46236-443-6,104.17.241.25,443,192.168.10.16,46236,6,7/7/2017 3:30,34,1,1,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,3.529412e+05,58823.52941,34.0,0.0,34.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,1,0,0,1.0,9.0,6.0,6.0,20,0,0,0,0,0,0,1,6,1,6,31,329,0,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,192.168.10.5-104.19.196.102-54863-443-6,104.19.196.102,443,192.168.10.5,54863,6,7/7/2017 3:30,3,2,0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,666666.66670,3.0,0.0,3.0,3.0,3.0,3.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,6.0,6.0,0.0,0.0,0,0,0,0,1,0,0,0,0.0,9.0,6.0,0.0,40,0,0,0,0,0,0,2,12,0,0,32,-1,1,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [9]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=[' Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]



In [10]:

# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])

# reformat Label
traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

# add new "is Attacker?" column
traffic[' is Attacker?'] = np.where(traffic[' Label'] == 0, 0, 1)


In [11]:
# ### drop benign traffics from hosts that send out attacking traffics

# # 1. find out host with attacking + benign traffics
# atker_benign = traffic[traffic.groupby([' Source IP'])[' Label'].transform('nunique') > 1]
# # 2. select benign traffic from those host
# atker_benign = atker_benign[atker_benign[' Label'] == 0]
# # 3. remove benign
# traffic = traffic.drop(atker_benign.index)



# traffic

In [12]:
columns = list(traffic.columns)
columns


[' Source IP',
 ' Source Port',
 ' Destination IP',
 ' Destination Port',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Fwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Coun

In [13]:
# create ip address dict {ip_addr: node_id}
src_ips = set(traffic[' Source IP'])
dst_ips = set(traffic[' Destination IP'])
ips = list(src_ips.union(dst_ips))

ips_dict = {ips[i]: i for i in range(len(ips))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(ips_dict)))


Number of nodes: 18598


In [14]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' Source IP'])[' Destination IP'].transform(len)
print(traffic)

              Source IP   Source Port  ...  is Attacker?  group_size
0        104.16.207.165           443  ...             0          29
1         104.16.28.216            80  ...             0         114
2         104.16.28.216            80  ...             0         114
3         104.17.241.25           443  ...             0           2
4        104.19.196.102           443  ...             0         120
...                 ...           ...  ...           ...         ...
2660372   192.168.10.14         51114  ...             0      129475
2660373   192.168.10.16         24054  ...             0      114034
2660374  23.208.163.130           443  ...             0          51
2660375   192.168.10.14         51694  ...             0      129475
2660376   192.168.10.14         57949  ...             0      129475

[2657447 rows x 77 columns]


In [15]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Label', None)
aggregations.pop(' Source IP', None)
aggregations.pop(' Source Port', None)
aggregations.pop(' Destination IP', None)
aggregations.pop(' Destination Port', None)
aggregations.pop(' Protocol', None)
aggregations.pop(' is Attacker?', None)

aggregations.update({' is Attacker?': lambda x: 1 if 1 in set(x) else 0})
print(aggregations)


{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [16]:
# low activity grouping
low_activities = traffic[traffic['group_size'] < 10]
low_activities = low_activities.groupby(' Source IP', as_index=False).agg(aggregations)
low_activities



,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,3.668571e+07,2.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,0.0,0.0,0.00000,0.0,0.0,0.0,0,0.000000,0.000000,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0,0.0,0.00,0,0,0.0,0.000000,0.0,0.0,0.000000,2.000000,0.0,0.000000,0.0,-1.00,-1.00,0.000000,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73,4.511675e+04,2.000000,0.250000,3.0,0.0,1.5,1.5,1.5,0.000000,0.0,0.0,0.0,0.0,8.720930e+03,4244.011538,2.269288e+04,31693.763375,4.510375e+04,2.820000e+02,4.511675e+04,4.511675e+04,0.000000,4.511675e+04,4.511675e+04,0.0,0.0,0.00000,0.0,0.0,0.0,0,58.000000,8.000000,4242.617935,1.393604,1.5,1.5,1.5,0.000000,0.0,0.0,0.0,0,0.0,1.0,0.25,0,0,0.0,2.250000,1.5,0.0,58.000000,2.000000,3.0,0.250000,0.0,4454.25,59.25,0.250000,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.193.219.21,6.480000e+02,3.000000,0.000000,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,6.480000e+02,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,0.0,0.0,0.00000,0.0,0.0,0.0,0,96.000000,0.000000,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.0,0.0,0.0,0,0.0,1.0,0.00,0,0,0.0,86.000000,86.0,0.0,96.000000,3.000000,258.0,0.000000,0.0,61.00,-1.00,1.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.193.219.24,1.661294e+05,2.000000,0.800000,258.0,0.0,258.0,0.0,129.0,182.433549,0.0,0.0,0.0,0.0,4.301911e+07,333556.025118,4.166945e+04,83140.944944,1.660928e+05,1.680000e+01,1.661180e+05,1.661180e+05,0.000000,1.661180e+05,1.661180e+05,165574.8,82787.4,117078.21534,165574.2,0.6,0.8,0,64.000000,16.000000,333481.472436,74.552682,0.0,258.0,137.6,140.230707,19969.2,0.0,0.8,0,0.0,1.0,0.00,0,0,0.2,199.520000,129.0,0.0,64.000000,2.000000,258.0,0.800000,0.0,65.00,-0.60,0.200000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.226.51.14,4.217540e+04,1.400000,0.600000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,15641.023359,4.217540e+04,0.000000,4.217540e+04,4.217540e+04,4.212860e+04,4.212860e+04,0.000000,4.212860e+04,4.212860e+04,0.0,0.0,0.00000,0.0,0.0,0.0,0,44.800000,19.200000,7824.395543,7816.627816,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0,1.0,0.60,0,0,0.6,0.000000,0.0,0.0,44.800000,1.400000,0.0,0.600000,0.0,155.20,866.00,0.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11180,98.158.54.105,2.570000e+02,3.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,11673.151750,1.285000e+

In [17]:
# attackers grouping
atk_activities = traffic[traffic[' is Attacker?'] == 1]
atk_activities = atk_activities.groupby(' Source IP', as_index=False).agg(aggregations)
atk_activities


,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,172.16.0.1,2.956412e+07,4.017171,3.225490,158.521657,5247.863025,114.118665,2.095020,24.556664,45.803378,2830.399745,1.865424,905.920939,1211.513215,317182.121404,96192.541960,2.907503e+06,8.457482e+06,2.862411e+07,225024.546124,2.933547e+07,5.546394e+06,1.159011e+07,2.853765e+07,545075.753076,8.864288e+06,1.918876e+06,3.528664e+06,8.584444e+06,349821.684346,0.012117,0,117.212187,94.934009,86743.368924,9449.173036,0.987842,2834.562531,431.105388,902.858431,2.013992e+06,0.106744,0.012117,0,0.459680,0.434348,0.007167,0,0,0.614946,472.879368,24.556664,905.920939,117.212187,4.017171,158.521657,3.225490,5247.863025,7331.240992,200.162452,1.512595,27.414352,103275.711488,19432.330451,116967.629362,89209.731133,2.719980e+07,1.509965e+06,2.835524e+07,2.612062e+07,1
1,192.168.10.12,3.013027e+07,5.000000,5.000000,161.000000,128.000000,161.000000,0.000000,17.888889,53.666667,128.000000,0.000000,14.222222,42.666667,4.800454,17.423337,1.799611e+06,2.450991e+06,5.128946e+06,28969.000000,2.560000e+07,3.198221e+06,2.634245e+06,5.100000e+06,117.000000,3.010000e+07,3.759477e+06,2.322569e+06,5.150000e+06,318.500000,0.000000,0,164.000000,164.000000,8.711669,8.711669,0.000000,161.000000,15.210526,45.891876,4.212129e+03,0.000000,0.000000,0,0.500000,0.500000,0.500000,0,0,1.000000,16.055556,17.888889,14.222222,164.000000,5.000000,161.000000,5.000000,128.000000,14718.500000,110.000000,0.500000,32.000000,31839.100000,11126.267980,51587.500000,25455.500000,5.100000e+06,1.747064e+04,5.100000e+06,5.050000e+06,1
2,192.168.10.14,4.012817e+05,4.755981,5.057416,4431.124402,107.736842,700.636364,0.000000,204.045950,198.270746,87.186603,2.009569,29.362901,44.737745,47960.846025,93.667237,7.676260e+04,1.150069e+05,2.302638e+05,177.090909,4.011073e+05,1.874357e+05,3.316208e+04,2.314064e+05,168935.693780,4.001571e+05,1.950188e+05,3.872412e+04,2.304383e+05,169237.483254,0.000000,0,113.818182,109.129187,46.028187,47.639050,0.000000,702.645933,89.350356,165.367536,2.719614e+05,0.000000,0.000000,0,1.000000,0.000000,0.000000,0,0,0.382775,94.335594,204.045950,29.362901,113.818182,4.755981,4431.124402,5.057416,107.736842,8192.000000,207.535885,3.086124,22.679426,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1
3,192.168.10.15,4.511831e+05,4.245902,4.325137,3593.650273,95.816940,523.346995,0.098361,154.350291,170.575954,79.360656,2.393443,27.850208,41.478106,175412.000815,23297.527461,8.735663e+04,1.287442e+05,2.533632e+05,198.658470,4.509802e+05,2.140238e+05,3.211725e+04,2.546007e+05,195742.969945,4.499535e+05,2.215904e+05,3.766137e+04,2.535668e+05,196000.587432,0.000000,0,104.371585,93.715847,11649.187520,11648.339941,0.098361,525.642077,69.494889,134.913609,2.915180e+05,0.000000,0.000000,0,0.983607,0.016393,0.000000,0,0,0.434426,74.007528,154.350291,27.850208,104.371585,4.245902,359

In [18]:
basic_flows = traffic.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,0,224.0.0.22,0,0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73,80,192.168.10.15,52772,6,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73,80,192.168.10.17,48301,6,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73,80,192.168.10.19,35264,6,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73,80,192.168.10.19,35275,6,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382955,99.192.248.32,80,192.168.10.5,49372,6,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1382956,99.192.248.32,80,192.168.10.5,49419,6,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20

In [19]:
activity_flows = basic_flows.groupby([' Source IP'], as_index=False)
activity_flows = activity_flows.agg(aggregations)

# reformat Source IP
# activity_flows = activity_flows.replace({" Source IP": ips_dict})
print(activity_flows)

atkers = activity_flows.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))



           Source IP   Flow Duration  ...    Idle Min   is Attacker?
0            0.0.0.0    3.668571e+07  ...  36700000.0              0
1          1.1.70.73    4.511675e+04  ...         0.0              0
2       1.193.219.21    6.480000e+02  ...         0.0              0
3       1.193.219.24    1.038322e+05  ...         0.0              0
4        1.226.51.14    4.217540e+04  ...         0.0              0
...              ...             ...  ...         ...            ...
16508  98.158.58.110    4.627100e+04  ...         0.0              0
16509  98.158.62.102    4.732533e+04  ...         0.0              0
16510  98.158.62.103    1.156000e+03  ...         0.0              0
16511  99.192.248.32    8.496417e+03  ...         0.0              0
16512  99.193.229.25    3.000000e+00  ...         0.0              0

[16513 rows x 71 columns]
Number of attackers: 10
Number of non_atkers 16503


In [20]:
# create low profiles dataframe
num_ips = len(ips_dict)
num_src_ips = len(activity_flows[' Source IP'])
num_noflow_ips = num_ips - num_src_ips


no_flow_prof_columns = columns[5:-3]
print(no_flow_prof_columns)
num_low_ips = round(0.98*num_noflow_ips)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_ips)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' is Attacker?'] = [0 for i in range(num_low_ips)]
low_profile_df[' Source IP'] = [ips[num_src_ips + i] for i in range(num_low_ips)]
low_profile_df



[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,-8.191412e+05,2.388897,1.092804,-451.884481,3.518916,140.447212,33.552497,79.854918,-37.061557,-0.267851,6.674477,3.557029,-0.628656,-4.483915e+06,110901.024295,-564212.726329,-489271.373734,1.080312e+06,5870.883807,4.729292e+05,-127346.278216,221844.433016,493491.106205,49649.440372,213037.606410,-13837.976364,28459.696259,-114413.629976,18369.444316,0.273505,0.0,33.701619,19.001416,-209816.585027,673.931894,-0.962876,-83.802839,-33.357874,60.536656,1528.125233,-0.040323,-0.040948,0.0,0.004725,1.001070,0.393874,0.0,0.0,0.588926,55.228621,-32.983273,-7.708047,47.877331,0.918732,-880.616709,0.206617,-12.232870,-1383.639321,-7500.605449,-0.277089,17.401979,24396.371461,16174.472003,16689.296224,23243.968620,476183.846950,-22528.718022,809161.470095,0,50.112.253.76
1,8.192952e+05,0.611103,0.407196,469.884481,-0.518916,-128.447212,-27.552497,-67.854918,37.061557,3.267851,-3.674477,-0.557029,0.628656,5.083915e+06,42945.129545,564346.726329,489271.373734,-1.080164e+06,-5779.550473,-4.729212e+05,127353.278216,-221844.433016,-493483.106205,-49644.297515,-213037.606410,13837.976364,-28459.696259,114413.629976,-18369.444316,-0.273505,0.0,42.298381,17.998584,293294.845897,19043.330011,6.962876,95.802839,45.357874,-60.536656,-1528.125233,0.040323,0.040948,0.0,-0.004725,0.998930,0.606126,0.0,0.0,0.411074,-37.228621,44.983273,10.708047,28.122669,2.081268,898.616709,1.293383,15.232870,1633.639321,7926.938783,0.610422,22.598021,-24396.371461,-16174.472003,-16689.296224,-23243.968620,-476183.846950,22528.718022,-809161.470095,0,52.1.96.98
2,1.139983e+06,2.264254,1.347970,-838.628678,-3.054317,-89.041052,-29.127464,10.727541,-6.994816,5.389925,-10.073736,-5.028932,0.779653,6.676802e+06,-109551.422374,248070.404028,667600.004967,1.068380e+06,337479.584757,1.011064e+06,639121.612221,588371.567923,-175888.877378,-286772.242705,-13455.499789,-26528.603500,-34314.098338,-84144.062058,10142.256412,0.277022,0.0,21.348748,19.895316,-163721.553728,-3828.640846,1.744473,-153.092669,-10.033325,65.440221,6588.102916,-0.043888,0.103118,0.0,-0.005541,1.002611,0.250162,0.0,0.0,0.461479,-53.935463,47.419281,-9.962759,27.094620,1.834263,747.281308,0.507094,16.892187,2674.473060,9665.569157,0.566675,17.638761,24347.442616,12904.638154,-48043.644245,4859.515317,-499255.501081,-55631.449655,-32078.153011,0,94.31.29.248
3,-1.139829e+06,0.735746,0.152030,856.628678,6.054317,101.041052,35.127464,1.272459,6.994816,-2.389925,13.073736,8.028932,-0.779653,-6.076802e+06,263397.576214,-247936.404028,-667600.004967,-1.068232e+06,-337388.251424,-1.011056e+06,-639114.612221,-588371.567923,175896.877378,286777.385562,13455.499789,26528.603500,34314.098338,84144.062058,-10142.256412,-0.277022,0.0,54.651252,17.104684,247199.814598,23545.902751,4.255527,165.092669,22.033325,-65.440221,-6588.102916,0.043888,-0.103118,0.0,0.005541,0.997389,0.74

In [21]:
# create attacker profiles dataframe

num_atk_ips = num_noflow_ips-num_low_ips

atker_prof = np.zeros((num_atk_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = atk_activities[attr].median()
  std = np.std(atk_activities[attr])
  atker_prof[:, col] = generate_random_array(median, std, num_atk_ips)


atker_profile_df = pd.DataFrame(atker_prof, columns = no_flow_prof_columns)
atker_profile_df[' is Attacker?'] = [1 for i in range(num_atk_ips)]
atker_profile_df[' Source IP'] = [ips[num_src_ips+num_low_ips + i] for i in range(num_atk_ips)]
atker_profile_df


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,2.920900e+06,22.463359,32.928800,10850.255103,166.084597,569.469268,1.058403,-10.259319,126.382251,601.210769,0.944741,-18.423570,-89.275143,82669.626771,-13108.331772,7.252787e+05,-1.065473e+05,4.490979e+06,-9161.587136,-4.951181e+06,1.221341e+06,5.641824e+03,4.304108e+06,181793.328256,3.611287e+06,8.400951e+05,1.091664e+06,3.741306e+06,143428.140873,-0.007234,0.0,-297.985178,656.498426,-7049.990271,743.354811,-0.334201,1031.172900,1.029724,-25.741256,-64372.052181,0.011152,-0.013826,0.0,1.026722,-0.008521,-0.254504,0.0,0.0,1.829363,99.828508,87.745028,-181.570759,-296.797194,-17.723650,4455.733603,10.796415,987.727414,8724.538336,661.867597,-11.815032,21.189432,-122721.210155,28393.968474,-145441.377183,-12437.950592,-4.045304e+06,277256.285235,-3.469285e+06,1,198.8.70.209
1,2.977620e+06,-12.936428,-22.538050,-7050.104830,53.284989,60.372752,-0.827299,159.010655,72.327223,-433.688971,3.411197,65.584460,171.517622,34942.184272,21637.297151,-3.040613e+05,1.148004e+06,4.483209e+05,9568.307058,5.859089e+06,-7.923236e+05,6.522188e+04,-3.789517e+06,143493.754905,2.277343e+06,-1.098493e+05,-1.380491e+05,9.699667e+05,182597.360079,0.007234,0.0,530.092658,-432.911139,11479.390614,3356.210225,0.565305,-78.866328,133.773989,273.655621,482198.503127,-0.011152,0.013826,0.0,0.329955,0.586191,0.264928,0.0,0.0,-0.618326,43.333887,61.006307,228.731649,528.904674,27.250580,-655.583329,-0.405665,-768.357828,7353.788082,-269.984475,15.808501,24.999169,137739.543488,-28393.968474,160459.710517,27456.283926,7.166575e+06,-277256.285235,7.543612e+06,1,185.19.219.27
2,-6.521806e+06,17.307494,2.871736,2831.387313,147.647872,599.131806,1.042096,11.647683,158.007309,324.610145,0.325091,40.565289,-0.509009,-14929.131501,-24398.212418,-5.596618e+05,4.934529e+05,9.892565e+06,62304.260936,-2.928986e+06,-1.040264e+06,1.226520e+06,6.109838e+06,320269.920177,5.403237e+06,1.227303e+06,1.876851e+05,3.816834e+06,215749.923863,0.016955,0.0,108.909718,600.805460,28026.288971,9822.424356,2.008828,807.119138,161.268626,124.295363,461619.904573,-0.008293,-0.013800,0.0,0.476708,0.592451,0.384198,0.0,0.0,0.691072,137.224928,145.852474,-142.844744,115.258961,-19.903868,-1830.779316,3.083283,-1055.793102,10194.415715,317.490092,23.780415,25.351857,-57331.173506,-58444.330076,119320.632937,94141.410671,2.666921e+06,54573.192991,-5.186898e+06,1,104.19.132.57
3,1.242033e+07,-7.780564,7.519013,968.762960,71.721713,30.710214,-0.810992,137.103653,40.702164,-157.088347,4.030847,6.595602,82.751488,132540.942545,32927.177797,9.808792e+05,5.480034e+05,-4.953266e+06,-61897.541013,3.836894e+06,1.469281e+06,-1.155656e+06,-5.595247e+06,5017.162984,4.853922e+05,-4.970571e+05,7.659301e+05,8.944390e+05,110275.577089,-0.016955,0.0,123.197763,-377.218173,-23596.888628,-5722.859321,-1.777724,145.187433,-26.464914,123.619002,-43793.453626,0.008293,0.0138

In [22]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)
activity_flows_all = activity_flows_all.append(atker_profile_df, ignore_index=True)




In [23]:

atkers = activity_flows_all.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows_all.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))
activity_flows_all


Number of attackers: 52
Number of non_atkers 18546


,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,3.668571e+07,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.054517,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.054517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.670000e+07,0.000000,3.670000e+07,36700000.0,0
1,1.1.70.73,4.511675e+04,2.000000,0.250000,3.000000,0.000000,1.500000,1.500000,1.500000,0.000000,0.000000,0.000000,0.000000,0.000000,8.720930e+03,4244.011538,2.269288e+04,3.169376e+04,4.510375e+04,2.820000e+02,4.511675e+04,4.511675e+04,0.000000e+00,4.511675e+04,4.511675e+04,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,58.000000,8.000000,4242.617935,1.393604,1.500000,1.500000,1.500000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.250000,0.0,0.0,0.000000,2.250000,1.500000,0.000000,58.000000,2.000000,3.000000,0.250000,0.000000,4454.250000,59.250000,0.250000,29.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
2,1.193.219.21,6.480000e+02,3.000000,0.000000,258.000000,0.000000,258.000000,0.000000,86.000000,148.956369,0.000000,0.000000,0.000000,0.000000,3.981481e+05,4629.629630,3.240000e+02,4.525483e+02,6.440000e+02,4.000000e+00,6.480000e+02,3.240000e+02,4.525483e+02,6.440000e+02,4.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.0,96.000000,0.000000,4629.629630,0.000000,0.000000,258.000000,64.500000,129.000000,16641.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.000000,86.000000,86.000000,0.000000,96.000000,3.000000,258.000000,0.000000,0.000000,61.000000,-1.000000,1.000000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
3,1.193.219.24,1.038322e+05,2.000000,0.500000,258.000000,0.000000,258.000000,0.000000,129.000000,182.433549,0.000000,0.000000,0.000000,0.000000,5.376194e+07,416805.849036,2.604478e+04,5.196309e+04,1.038094e+05,1.187500e+01,1.038251e+05,1.038251e+05,0.000000e+00,1.038251e+05,1.038251e+05,1.034842e+05,5.174212e+04,73173.884587,1.034839e+05,0.375000,0.875000,0.0,64.000000,10.000000,416759.253610,46.595426,0.000000,258.000000,150.500000,143.502831,20801.250000,0.000000,0.875000,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.125000,221.450000,129.000000,0.000000,64.000000,2.000000,258.000000,0.500000,0.000000,65.000000,-0.750000,0.125000,32.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
4,1.226.51.14,4.217540e+04,1.400000

In [24]:
#activity_flows_all.to_csv('ips_benign_removed.csv', index=False)


In [25]:
activity_flows_all = activity_flows_all.sort_values(by=' Source IP', ignore_index=True)
activity_flows_all


,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,3.668571e+07,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.054517,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000e+00,3.670000e+07,3.670000e+07,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,0.000000,0.000000,0.054517,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.670000e+07,0.000000,3.670000e+07,36700000.0,0
1,0.40.115.17,4.081536e+05,1.435509,0.780510,1182.406255,13.784396,-91.294832,26.979362,-1.253724,-64.733738,-12.901163,11.887545,-9.983522,0.778984,-7.245948e+06,292316.234914,3.418549e+05,-4.776727e+05,7.830626e+05,1.093897e+05,1.961209e+05,-2.327103e+05,9.614026e+04,-1.058757e+06,-1.165853e+05,2.425709e+05,47948.516139,-3.690806e+04,1.088119e+05,-12649.642837,0.277938,0.0,18.458960,8.089396,292713.989255,37797.251905,6.215192,-125.326997,53.833597,52.311007,-88616.313506,0.028616,-0.310478,0.0,0.019930,1.032503,0.167543,0.0,0.0,0.218917,-27.698146,14.916297,-12.443730,60.113497,0.858982,1055.311652,0.422484,17.473736,-2693.140768,2999.973802,0.435263,15.319842,18905.657131,8113.879288,20379.185385,-46229.935357,-2.008696e+05,-24643.370482,4.819933e+04,NaN,0
2,0.40.124.75,2.027540e+05,0.974610,1.229107,45.061777,1.940236,56.825645,39.539235,28.610622,65.369819,6.330175,20.292216,-9.900285,-0.455214,-4.781149e+06,-109312.827951,2.245550e+04,-4.952081e+05,3.409015e+05,9.648591e+04,1.239638e+06,-2.197262e+05,-4.393995e+05,-2.638738e+04,3.619226e+05,7.577715e+03,90731.988175,-3.137407e+03,-7.901931e+04,59896.582432,-0.224704,0.0,39.197578,25.910383,55153.314914,13894.891486,-0.514922,65.849015,-43.720547,3.321144,27432.708312,-0.033530,-0.000110,0.0,-0.016438,1.035415,0.696676,0.0,0.0,0.827328,-18.878786,-19.746592,2.243801,25.781542,0.981539,-533.080210,0.636833,16.566698,-3227.417930,2072.895246,0.823843,19.346842,13231.216045,-21225.847792,62702.567754,41346.453424,9.533485e+05,48436.699065,-6.796662e+05,NaN,0
3,0.40.184.9,1.118174e+06,2.472943,0.701413,859.530682,5.975055,-97.333370,-19.832984,-72.119541,45.719884,11.517931,-16.723005,17.160074,-0.190368,2.837747e+06,282596.019739,3.275188e+05,-5.289222e+05,-8.704758e+05,-2.369404e+05,2.447567e+05,3.450481e+05,-3.964938e+05,-7.516118e+05,-2.426735e+05,-1.969001e+05,7081.947740,1.591422e+04,1.237555e+05,11579.007192,0.182054,0.0,20.822627,20.491038,96031.114578,86.665060,0.580298,171.500346,66.072859,6.048205,44059.192512,-0.045800,-0.113217,0.0,-0.006324,1.033642,0.783732,0.0,0.0,0.883283,-36.717159,80.014613,-3.304388,14.552742,0.742693,-1185.197877,0.250139,0.1

In [26]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]


In [27]:
# create hypergraph
connection = traffic.groupby([' Source IP', ' Destination IP'], as_index=False)
connection = connection.agg({' Flow Duration' : 'count'})
src_ip_ids = [ips_dict[ip] for ip in connection[' Source IP']]
dst_ip_ids = [ips_dict[ip] for ip in connection[' Destination IP']]
edges = list(zip(src_ip_ids, dst_ip_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [29]:
# generate features
# features = X
# features = np.nan_to_num(features)
# features = preprocessing.normalize(features)
# features = csr_matrix(features)

# generate features from feature selection set
X = X.fillna(0)
selected_features = selectFeatures(X, Y)
pd.DataFrame(activity_flows_all, columns=selected_features).to_csv('37_feat_50_attackers.csv')
print(f'Selected Features "from All": {selected_features}')
features = X[selected_features]
print("Shape of features: ", features.shape)

features = preprocessing.normalize(features)
features = csr_matrix(features)


[Training] RandomForest
>>> Training Time: 0:00:01.608447
>>> Training Accuracy : 99.93548387096774

[Training] ExtraTrees
>>> Training Time: 0:00:00.287667
>>> Training Accuracy : 99.93548387096774

[Training] DecisionTree
>>> Training Time: 0:00:00.101289
>>> Training Accuracy : 99.93548387096774

Selected Features "from All": ['Idle Mean', 'Total Length of Fwd Packets', 'Subflow Fwd Packets', ' Flow IAT Mean', 'Bwd Packet Length Max', ' Flow Packets/s', ' Bwd IAT Mean', ' Max Packet Length', 'Flow Bytes/s', ' Fwd Packet Length Mean', ' Total Length of Bwd Packets', ' min_seg_size_forward', ' URG Flag Count', ' ACK Flag Count', ' Bwd IAT Max', ' Avg Bwd Segment Size', ' Bwd IAT Std', ' Bwd Packet Length Mean', ' Idle Max', ' Bwd IAT Min', ' Total Fwd Packets', 'Fwd IAT Total', ' Bwd Packet Length Min', ' Bwd Packet Length Std', ' act_data_pkt_fwd', ' Total Backward Packets', ' Fwd Packet Length Std', ' Fwd Packet Length Max', ' Bwd Packets/s', ' Idle Std', ' Bwd Header Length', 'Bwd 

In [34]:
# generate labels
labels = Y

In [38]:
# train/test split

splits = {}
attackers_ids = list(atkers.index)
V = len(ips)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set


In [39]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] == 1:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] == 1:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  40
Number of atker in test set:  12


In [40]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()